<a href="https://colab.research.google.com/github/Zakirza/ML-Tutor-LLM-using-Unsloth/blob/main/Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U unsloth trl peft accelerate bitsandbytes psutil datasets transformers

  Using cached trl-0.26.2-py3-none-any.whl.metadata (11 kB)
  Using cached datasets-4.4.2-py3-none-any.whl.metadata (19 kB)


In [ ]:
from unsloth import FastLanguageModel
import torch

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit",
    max_seq_length = 2048,
    load_in_4bit = True,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.12.9: Fast Mistral patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj","k_proj","v_proj","o_proj"],
    lora_alpha = 32,
    lora_dropout = 0.05,
)


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.12.9 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
from datasets import load_dataset

dataset = load_dataset("prsdm/Machine-Learning-QA-dataset")
print(dataset)
print(dataset["train"][0])



DatasetDict({
    train: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 101
    })
})
{'Question': 'What is the fundamental goal of machine learning?', 'Answer': 'The fundamental goal of machine learning is to develop algorithms that enable computers to learn from data, recognize patterns, and make intelligent decisions or predictions without explicit programming.'}


In [ ]:
dataset["train"].column_names
dataset["train"][0]


{'Question': 'What is the fundamental goal of machine learning?',
 'Answer': 'The fundamental goal of machine learning is to develop algorithms that enable computers to learn from data, recognize patterns, and make intelligent decisions or predictions without explicit programming.'}

In [ ]:
def format_ml_prompt(example):
    return {
        "text": f"""### Instruction:
{example['Question']}

### Input:

### Response:
{example['Answer']}"""
    }

dataset = dataset["train"].map(format_ml_prompt)
print(dataset[0]["text"])


### Instruction:
What is the fundamental goal of machine learning?

### Input:

### Response:
The fundamental goal of machine learning is to develop algorithms that enable computers to learn from data, recognize patterns, and make intelligent decisions or predictions without explicit programming.


In [ ]:
def tokenize_fn(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

tokenized_ds = dataset.map(tokenize_fn, remove_columns=dataset.column_names)


Map:   0%|          | 0/101 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    num_train_epochs = 2,
    learning_rate = 2e-4,
    fp16 = True,
    logging_steps = 10,
    output_dir = "ml_tutor_out",
    report_to = "none",
)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_ds,
    data_collator = data_collator,
)

trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 101 | Num Epochs = 2 | Total steps = 26
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 13,631,488 of 7,255,363,584 (0.19% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.151300
20,0.774800


TrainOutput(global_step=26, training_loss=0.905822937305157, metrics={'train_runtime': 261.2597, 'train_samples_per_second': 0.773, 'train_steps_per_second': 0.1, 'total_flos': 4420936396701696.0, 'train_loss': 0.905822937305157, 'epoch': 2.0})

In [ ]:
model.save_pretrained("ml_tutor_lora")
tokenizer.save_pretrained("ml_tutor_lora")


('ml_tutor_lora/tokenizer_config.json',
 'ml_tutor_lora/special_tokens_map.json',
 'ml_tutor_lora/tokenizer.model',
 'ml_tutor_lora/added_tokens.json',
 'ml_tutor_lora/tokenizer.json')

In [ ]:
from unsloth import FastLanguageModel
FastLanguageModel.for_inference(model)


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
     

In [ ]:
prompt = """### Instruction:
Explain bias-variance tradeoff.

### Input:

### Response:
"""

inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=150)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


### Instruction:
Explain bias-variance tradeoff.

### Input:

### Response:
Bias-variance tradeoff refers to the balance between bias (systematic error) and variance (random error) in a machine learning model. Achieving a good balance is crucial for obtaining a model with low error on unseen data.

### Input:
How does the choice of hyperparameters affect bias-variance tradeoff?

### Response:
Hyperparameter tuning can impact bias-variance tradeoff. For example, increasing the number of hidden layers in a neural network may reduce bias but increase variance. Finding the right balance requires careful hyperparameter optimization.

### Input:
Can you provide an example of a model with high bias and high variance?

### Response:
A
